# Technical Setup



## Import of Demo Translations
In the first step we import the given translations as pandas Dataframes and print a quick overview of the dataframe.


In [ ]:
import pickle

from mlflow import transformers

with open('machine_translation.pkl', 'rb') as f:
    # Einlesen des Dataframes
    data = pickle.load(f)

print(data)

## Import of AI-Models
In the second step we import the AI-Models which are given in the specified task. For doing so we use the `llama-cpp-python` library (further documentation can be found [here](https://github.com/abetlen/llama-cpp-python)) and import the models directly from [huggingface](https://huggingface.co/).

Quick overview and installation guide of llama.cpp: 
- https://www.datacamp.com/tutorial/llama-cpp-tutorial
- https://christophergs.com/blog/running-open-source-llms-in-python



In [ ]:
from llama_cpp import Llama

gemma = Llama.from_pretrained(
	repo_id="lmstudio-ai/gemma-2b-it-GGUF",
	filename="gemma-2b-it-q8_0.gguf",
    n_gpu_layers=1,
    verbose=False,
)


In [ ]:
llama32 = Llama.from_pretrained(
	repo_id="hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF",
	filename="llama-3.2-3b-instruct-q8_0.gguf",
    n_gpu_layers=1,
    verbose=False,)


In [ ]:
llama31 = Llama.from_pretrained(
	repo_id="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
	filename="Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf",
    verbose=False,)


In [ ]:
aya23 = Llama.from_pretrained(
	repo_id="bartowski/aya-23-35B-GGUF",
	filename="aya-23-35B-Q5_K_M.gguf",
    n_gpu_layers=1,
    verbose=False,
)


In [ ]:
output = llama31(
	"Q: Übersetze mir folgenden Text ins Englische: 'Hallo ich heiße Peter.'",
	max_tokens=512,
	echo=False,
    stop=["Q:"]
)

# print(output)
print(output['choices'][0]['text'])

In [ ]:
output2 = aya23(
	"Q: Übersetze mir folgenden Text ins Englische: 'Hallo ich heiße Peter.'",
	max_tokens=512,
	echo=False,
    stop=["Q:"]
)

# print(output)
print(output2['choices'][0]['text'])

In [ ]:
print(output)

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("check-localhost-connection")

with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("bar", 3)

## MetricX Score berechnen
https://github.com/google-research/metricx

In [11]:
import subprocess
import json
import os

def calculate_metricx_score(source, reference, hypothesis):
    """
    Calculates the MetricX-score based on source, reference, and hypothesis using metricx24.
    We are currently using the metricx-24-hybrid-large-v2p6-bfloat16 model but there are also other options
        as can be seen here: https://github.com/google-research/metricx

    Args:
        source: The source text (String).
        reference: The reference translation (String).
        hypothesis: The hypothesis translation (String).

    Returns:
        The calculated score as a float or None in case of an error.
    """

    data = [{"id": "1", "source": source, "reference": reference, "hypothesis": hypothesis}]

    # Create temporary JSONL files
    input_file = "./temp_input.jsonl"
    output_file = "./temp_output.jsonl"
    model = "google/metricx-24-hybrid-large-v2p6-bfloat16"

    try:
        with open(input_file, "w", encoding="utf-8") as f:
            for entry in data:
                json.dump(entry, f)
                f.write("\n")

        command = [
            "python", "-m", "metricx24.predict",
            "--tokenizer", "google/mt5-xl",
            "--model_name_or_path", model,
            "--max_input_length", "1536",
            "--batch_size", "1",
            "--input_file", input_file,
            "--output_file", output_file
        ]

        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        # Capture output and errors (optional, can be useful for debugging)
        # for line in process.stdout:
        #     print(line, end="")
        # for line in process.stderr:
        #     print(f"ERROR: {line}", end="")

        process.wait()

        if process.returncode != 0:
             print(f"Error executing metricx24. Return code: {process.returncode}")
             return None

        # Read score from the output file
        with open(output_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    output_data = json.loads(line)
                    score = float(output_data.get("prediction"))  # Ensure that "score" exists
                    return score
                except (json.JSONDecodeError, ValueError, AttributeError):
                    print("Error parsing the output file.")
                    return None

        return None # If no valid line was found in the output file

    finally:
        # Remove temporary files
        try:
            os.remove(input_file)
            os.remove(output_file)
        except FileNotFoundError:
            pass #If the files don't exist for some reason, the error is caught


In [12]:
# Example call
source_text = "I am learning Python for Machine Learning."
reference_text = "I am learning Python for machine learning."
hypothesis_text = "I'm studying Python for machine learning."

score = calculate_metricx_score(source_text, reference_text, hypothesis_text)
print(score)

1.1640625


In [ ]:
## zweite Möglichkeit metricx zu nutzen...
import sys
from metricx24 import predict

# Simuliere Kommandozeilenargumente
sys.argv = [
    "predict.py",  # Name des Skripts (wird ignoriert)
    "--tokenizer", "google/mt5-xl",
    "--model_name_or_path", "google/metricx-24-hybrid-large-v2p6-bfloat16",
    "--max_input_length", "1536",
    "--batch_size", "1",
    "--input_file", "./input.jsonl",
    "--output_file", "./output.jsonl"
]

# Rufe die main-Funktion auf
predict.main()

print("Vorhersage abgeschlossen. Ergebnisse sind in 'output.jsonl' gespeichert.")